In [1]:
import nmm
from qutip import sigmaz,sigmax,sigmay,Qobj
import numpy as np

/home/mcditoos/anaconda3/envs/cumulant/lib/python3.12/site-packages/qutip/__init__.py:66: UserWarning: The new version of Cython, (>= 3.0.0) is not supported.
  warnings.warn(


In [2]:
bath=nmm.OverdampedBath(coupling=0.05,cutoff=5,T=1)

In [3]:
w,w1=1,2
t=np.linspace(0,10,100)

In [33]:
cc=nmm.cumulant.csolve(Hsys=sigmaz()/2,baths=[bath,bath],t=t,Qs=[(sigmax()+sigmaz()),sigmay()],eps=1e-6,cython=False)
cc_c=nmm.cumulant.csolve(Hsys=sigmaz()/2,baths=[bath,bath],t=t,Qs=[(sigmax()+sigmaz()),sigmay()],eps=1e-6,cython=True)

In [34]:
ans=cc_c.jump_operators(cc_c.Qs[0])

In [36]:
cons=cc_c.generator2(False)

Calculating Integrals ...: 100%|██████████| 4/4 [00:01<00:00,  3.79it/s]


In [37]:
len(cons)

100

In [38]:
cc_c.generator()

Generating time dependence...: 100%|██████████| 100/100 [00:00<00:00, 2149.89it/s]


In [40]:
dd=[np.isclose((cc_c.generators[i]-cons[i]).full(),np.zeros(cc_c.generators[0].full().shape)).all() for i in range(len(cons))]
np.array(dd).all()

True

In [41]:
w0=1
Deltaw=np.pi*1e-1
alpha=0.05
T=1
gamma=5
s=1
tf=1/(alpha)
eps=1e-14
Hsys=Qobj([[w0+Deltaw,0,0],[0,w0-Deltaw,0],[0,0,0]])
Q=Qobj([[0,0,1],[0,0,1],[1,1,0]])

In [42]:
cc=nmm.cumulant.csolve(Hsys=Hsys,baths=[bath,bath],t=t,Qs=[Q,Q],eps=1e-6,cython=True)


In [44]:
cons=cc.generator2(False)

Calculating Integrals ...: 100%|██████████| 16/16 [00:02<00:00,  6.28it/s]


In [45]:
cc.generator()

Generating time dependence...: 100%|██████████| 100/100 [00:00<00:00, 432.26it/s]


In [48]:
dd=[np.isclose((cc.generators[i]-cons[i]).full(),np.zeros(cc.generators[0].full().shape)).all() for i in range(len(cons))]
np.array(dd).all()

True

In [24]:
%lprun -f cc_c.generator w_cython=cc_c.generator()

Calculating Integrals ...:   0%|          | 0/9 [00:00<?, ?it/s]/home/mcditoos/miniconda3/envs/nm/lib/python3.12/site-packages/scipy/integrate/_quadpack_py.py:446: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  re_retval = quad(refunc, a, b, args, full_output, epsabs,
Calculating Integrals ...:  11%|█         | 1/9 [00:02<00:19,  2.44s/it]/home/mcditoos/miniconda3/envs/nm/lib/python3.12/site-packages/scipy/integrate/_quadpack_py.py:449: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to ana

Timer unit: 1e-09 s

Total time: 21.7329 s
File: /home/mcditoos/github/NonMarkovianMethods/nmm/cumulant/cumulant.py
Function: generator at line 142

Line #      Hits         Time  Per Hit   % Time  Line Contents
   142                                               def generator(self, approximated=False):
   143         1       3537.0   3537.0      0.0          if type(self.Hsys) != np.ndarray:
   144         1     915913.0 915913.0      0.0              evals, all_state = self.Hsys.eigenstates()
   145                                                   else:
   146                                                       evals, all_state = np.linalg.eig(self.Hsys)
   147                                                       all_state = [i.reshape((len(i), 1)) for i in all_state]
   148         1        325.0    325.0      0.0          generators=[]
   149         2       3195.0   1597.5      0.0          for Q,bath in zip(self.Qs,self.baths):
   150         1        265.0    265.0      0.0

In [13]:
def population(den, a, b):
    return [den[i][a, b] for i in range(len(den))]

In [31]:
import sys
import numpy as np
from numba import jit





class SemiInfiniteFunc:
    """
    Argument transform from (start, +-oo) to (0, 1)
    """
    def __init__(self, func, start, infty):
        self._func = func
        self._start = start
        self._sgn = -1 if infty < 0 else 1
        # Overflow threshold for the 1/t**2 factor
        self._tmin = sys.float_info.min**0.5

    def get_t(self, x):
        z = self._sgn * (x - self._start) + 1
        if z == 0:
            # Can happen only if point not in range
            return np.inf
        return 1 / z

    def __call__(self, t):
        if t < self._tmin:
            return 0.0
        else:
            x = self._start + self._sgn * (1 - t) / t
            f = self._func(x)
            return self._sgn * (f / t) / t




def _quadrature_gk(a, b, f, norm_func, x, w, v):
    """
    Generic Gauss-Kronrod quadrature
    """

    fv = [0.0]*len(x)

    c = 0.5 * (a + b)
    h = 0.5 * (b - a)

    # Gauss-Kronrod
    s_k = 0.0
    s_k_abs = 0.0
    for i in range(len(x)):
        ff = f(c + h*x[i])
        fv[i] = ff

        vv = v[i]

        # \int f(x)
        s_k += vv * ff
        # \int |f(x)|
        s_k_abs += vv * abs(ff)

    # Gauss
    s_g = 0.0
    for i in range(len(w)):
        s_g += w[i] * fv[2*i + 1]

    # Quadrature of abs-deviation from average
    s_k_dabs = 0.0
    y0 = s_k / 2.0
    for i in range(len(x)):
        # \int |f(x) - y0|
        s_k_dabs += v[i] * abs(fv[i] - y0)

    # Use similar error estimation as quadpack
    err = float(norm_func((s_k - s_g) * h))
    dabs = float(norm_func(s_k_dabs * h))
    if dabs != 0 and err != 0:
        err = dabs * min(1.0, (200 * err / dabs)**1.5)

    eps = sys.float_info.epsilon
    round_err = float(norm_func(50 * eps * h * s_k_abs))

    if round_err > sys.float_info.min:
        err = max(err, round_err)

    return h * s_k, err, round_err


def _quadrature_gk15(a, b, f, norm_func):
    """
    Gauss-Kronrod 15 quadrature with error estimate
    """
    # Gauss-Kronrod points
    x = (0.991455371120812639206854697526329,
         0.949107912342758524526189684047851,
         0.864864423359769072789712788640926,
         0.741531185599394439863864773280788,
         0.586087235467691130294144838258730,
         0.405845151377397166906606412076961,
         0.207784955007898467600689403773245,
         0.000000000000000000000000000000000,
         -0.207784955007898467600689403773245,
         -0.405845151377397166906606412076961,
         -0.586087235467691130294144838258730,
         -0.741531185599394439863864773280788,
         -0.864864423359769072789712788640926,
         -0.949107912342758524526189684047851,
         -0.991455371120812639206854697526329)

    # 7-point weights
    w = (0.129484966168869693270611432679082,
         0.279705391489276667901467771423780,
         0.381830050505118944950369775488975,
         0.417959183673469387755102040816327,
         0.381830050505118944950369775488975,
         0.279705391489276667901467771423780,
         0.129484966168869693270611432679082)

    # 15-point weights
    v = (0.022935322010529224963732008058970,
         0.063092092629978553290700663189204,
         0.104790010322250183839876322541518,
         0.140653259715525918745189590510238,
         0.169004726639267902826583426598550,
         0.190350578064785409913256402421014,
         0.204432940075298892414161999234649,
         0.209482141084727828012999174891714,
         0.204432940075298892414161999234649,
         0.190350578064785409913256402421014,
         0.169004726639267902826583426598550,
         0.140653259715525918745189590510238,
         0.104790010322250183839876322541518,
         0.063092092629978553290700663189204,
         0.022935322010529224963732008058970)

    return _quadrature_gk(a, b, f, norm_func, x, w, v)


def _quadrature_gk21(a, b, f, norm_func):
    """
    Gauss-Kronrod 21 quadrature with error estimate
    """
    # Gauss-Kronrod points
    x = (0.995657163025808080735527280689003,
         0.973906528517171720077964012084452,
         0.930157491355708226001207180059508,
         0.865063366688984510732096688423493,
         0.780817726586416897063717578345042,
         0.679409568299024406234327365114874,
         0.562757134668604683339000099272694,
         0.433395394129247190799265943165784,
         0.294392862701460198131126603103866,
         0.148874338981631210884826001129720,
         0,
         -0.148874338981631210884826001129720,
         -0.294392862701460198131126603103866,
         -0.433395394129247190799265943165784,
         -0.562757134668604683339000099272694,
         -0.679409568299024406234327365114874,
         -0.780817726586416897063717578345042,
         -0.865063366688984510732096688423493,
         -0.930157491355708226001207180059508,
         -0.973906528517171720077964012084452,
         -0.995657163025808080735527280689003)

    # 10-point weights
    w = (0.066671344308688137593568809893332,
         0.149451349150580593145776339657697,
         0.219086362515982043995534934228163,
         0.269266719309996355091226921569469,
         0.295524224714752870173892994651338,
         0.295524224714752870173892994651338,
         0.269266719309996355091226921569469,
         0.219086362515982043995534934228163,
         0.149451349150580593145776339657697,
         0.066671344308688137593568809893332)

    # 21-point weights
    v = (0.011694638867371874278064396062192,
         0.032558162307964727478818972459390,
         0.054755896574351996031381300244580,
         0.075039674810919952767043140916190,
         0.093125454583697605535065465083366,
         0.109387158802297641899210590325805,
         0.123491976262065851077958109831074,
         0.134709217311473325928054001771707,
         0.142775938577060080797094273138717,
         0.147739104901338491374841515972068,
         0.149445554002916905664936468389821,
         0.147739104901338491374841515972068,
         0.142775938577060080797094273138717,
         0.134709217311473325928054001771707,
         0.123491976262065851077958109831074,
         0.109387158802297641899210590325805,
         0.093125454583697605535065465083366,
         0.075039674810919952767043140916190,
         0.054755896574351996031381300244580,
         0.032558162307964727478818972459390,
         0.011694638867371874278064396062192)

    return _quadrature_gk(a, b, f, norm_func, x, w, v)


def _gauss_kron(f,a,b,tol=1e-6):
    integral,err,rnd=_quadrature_gk15(a,b,f,np.linalg.norm)  
    if abs(err)<tol:
        q= integral
    else:
        c=(a+b)/2
        qa= _gauss_kron(f,a,c,tol)
        qb=_gauss_kron(f,c,b,tol)   
        q=qa+qb 
    return q

def gauss_kron(f,a,b,tol=1e-6,args=()):
    func= lambda x: f(x,*args)
    if (a==0) and (b ==np.infty):
        func2=SemiInfiniteFunc(func,0,np.infty)
        b=1
    return _gauss_kron(func2,a,b,tol)

def quad_gauss(f,a,b,tol=1e-6,args=()):
    if isinstance(args[-1],np.ndarray):
        params=args[:-1]
        return [gauss_kron(f,a,b,tol,args=(*params,i)) for i in args[-1]]
    else:
        return gauss_kron(f,a,b,tol,args=args)

import functools

In [293]:
import jax.numpy as jnp
from jax import jit
from jax.scipy.linalg import expm
from flax import struct

In [294]:
from typing import Any

In [376]:

class Qobj:
    def __init__(self,op):
        self.data=op
        self.shape=op.shape
        self.dtype=op.dtype
    def __add__(self,other):
        if isinstance(other,Qobj):
            return Qobj(self.data+other.data)
        if other==0:
            return self
        else:
            return TypeError(f"Not Implemented for {type(other)}")
    def __radd__(self,other):
        if isinstance(other,Qobj):
            return Qobj(self.data+other.data)
        if other==0:
            return self
        else:
            return TypeError(f"Not Implemented for {type(other)}")
    def __sub__(self,other):
        if isinstance(other,Qobj):
            return Qobj(self.data-other.data)
        if other==0:
            return self
        else:
            return TypeError(f"Not Implemented for {type(other)}")
    def __rsub__(self,other):
        if isinstance(other,Qobj):
            return Qobj(self.data-other.data)
        if other==0:
            return self
        else:
            return TypeError(f"Not Implemented for {type(other)}")
    def dag(self):
        return Qobj(jnp.conjugate(jnp.transpose(self.data)))
    def __str__(self):
        s=f"Operator: \n {self.data}"
        return s
    def __repr__(self):
        return self.__str__()
    def eigenstates(self):
        eigvals,eigvecs=jnp.linalg.eig(self.data)
        eigvals=eigvals.tolist()
        eigvecs = [i.reshape((len(i), 1)) for i in eigvecs]
        eigvecs=[Qobj(i) for i in eigvecs]
        return eigvals,eigvecs
    def __mul__(self,other):
        if (isinstance(other,Number)):
            return Qobj(self.data*other)
        return Qobj(self.data @ other.data)
    def __rmul__(self,other):
        if (isinstance(other,Number)):
            return Qobj(self.data * other)
        return Qobj(other.__mul__(self))
    def expm(self):
        return expm(self.data)
        
        
    

In [377]:
sp=Qobj(jnp.array([[0,0],[1,0]]))
sm=Qobj(jnp.array([[0,1],[0,0]]))
sz=Qobj(jnp.array([[1,0],[0,-1]]))

In [378]:
class spre:
    def __init__(self, op):
        op=op.data
        self.right = jnp.kron(op, jnp.eye(op.shape[0]))
        self.dim = op.shape[0]
        ## it may be worth using tensor contractions here (einsum)
        self.func = lambda x: Qobj((self.right@x.data.reshape(self.dim**2)).reshape(self.dim, self.dim))

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

    def __add__(self, other):
        self.right += other.right
        self.func = lambda x: Qobj((self.right@x.data.reshape(self.dim**2)).reshape(self.dim, self.dim))
        return self

    def __radd__(self, other):
        self.__add__(other)
        
    def __sub__(self, other):
        if isinstance(other,(spre,spost)):
            self.right -= other.right
            self.func = lambda x: Qobj((self.right@x.data.reshape(self.dim**2)).reshape(self.dim, self.dim))
            return self
        else:
            return TypeError
    def __rsub__(self,other):
        self.__sub__(other)

    def __mul__(self, other):
        if type(other) in (int, float, complex, jnp.complex128):
            self.right *= other
            self.func = lambda x: Qobj((self.right@x.data.reshape(self.dim**2)).reshape(self.dim, self.dim))
            return self
        self.right = self.right @ other.right
        self.func = lambda x: Qobj((self.right@x.data.reshape(self.dim**2)).reshape(self.dim, self.dim))
        return self
    def __rmul__(self,other):
        if type(other) in (int, float, complex, jnp.complex128):
            self.right *= other
            self.func = lambda x: Qobj((self.right@x.data.reshape(self.dim**2)).reshape(self.dim, self.dim))
            return self
        else:
            self.__mul__(other)
class spost:
    def __init__(self, op):
        op=op.data
        self.right = jnp.kron(jnp.eye(op.shape[0]), op.T)
        self.dim = op.shape[0]
        ## it may be worth using tensor contractions here (einsum)
        self.func = lambda x: Qobj((self.right@x.data.reshape(self.dim**2)).reshape(self.dim, self.dim))

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

    def __add__(self, other):
        self.right += other.right
        self.func = lambda x: Qobj((self.right@x.data.reshape(self.dim**2)).reshape(self.dim, self.dim))
        return self

    def __radd__(self, other):
        self.__add__(other)
        
    def __sub__(self, other):
        if isinstance(other,(spre,spost)):
            self.right -= other.right
            self.func = lambda x: Qobj((self.right@x.data.reshape(self.dim**2)).reshape(self.dim, self.dim))
            return self
        else:
            return TypeError
    def __rsub__(self,other):
        self.__sub__(other)
    def __mul__(self, other):
        if type(other) in (int, float, complex, jnp.complex128):
            self.right *= other
            self.func = lambda x: Qobj((self.right@x.data.reshape(self.dim**2)).reshape(self.dim, self.dim))
            return self
        self.right = self.right @ other.right
        self.func = lambda x: Qobj((self.right@x.data.reshape(self.dim**2)).reshape(self.dim, self.dim))
        return self
    def __rmul__(self,other):
        if type(other) in (int, float, complex, jnp.complex128):
            self.right *= other
            self.func = lambda x: Qobj((self.right@x.data.reshape(self.dim**2)).reshape(self.dim, self.dim))
            return self
        else:
            self.__mul__(other)


In [379]:
from numbers import Number

In [381]:
def test(sp,sm,sz):
    return (spre(sp) * spre(sm) -(spre(sp*sm) + spre(sp * sm)))(sz)

In [382]:
test(sp,sm,sz)

Operator: 
 [[0. 0.]
 [0. 1.]]

In [383]:
Hsys=sz*0.5
j=1
k=0
Q=sp+sm
evals, all_state = Hsys.eigenstates()

In [384]:
Q

Operator: 
 [[0 1]
 [1 0]]

In [385]:
from collections import defaultdict

In [386]:
def jump_operators(Hsys,Q):
        evals, all_state = Hsys.eigenstates()
        N=len(all_state)
        collapse_list = []
        ws = []
        for j in range(N):
            for k in range(j + 1, N):
                Deltajk = evals[k] - evals[j]
                ws.append(Deltajk)
                collapse_list.append(
                    (
                        all_state[j]
                        * all_state[j].dag()
                        * Q
                        * all_state[k]
                        * all_state[k].dag()
                    )
                )  # emission
                ws.append(-Deltajk)
                collapse_list.append(
                    (
                        all_state[k]
                        * all_state[k].dag()
                        * Q
                        * all_state[j]
                        * all_state[j].dag()
                    )
                )  # absorption
        collapse_list.append(Q - sum(collapse_list))  # Dephasing
        ws.append(0)
        output = defaultdict(list)
        for k,key in enumerate(ws):
            output[key].append(collapse_list[k])
        eldict={x:sum(y) for x, y in output.items()}
        dictrem = {}
        empty = 0*Qobj(Hsys)# Discarding is not working
        for keys, values in eldict.items():
            if (values != empty):
                dictrem[keys] = values
        return dictrem

In [387]:
jumps=jump_operators(Hsys,Q)

In [388]:
from tqdm import tqdm

In [389]:
def matrix_form(jumps,combinations):   
    matrixform=[]    
    for i in combinations:
            matrixform.extend((i,[spre(jumps[i[1]]) * spost(jumps[i[0]].dag()) 
                            -(0.5 *(spre(jumps[i[0]].dag() * jumps[i[1]]) 
                            + spost(jumps[i[0]].dag() * jumps[i[1]])))]))
    return matrixform

In [390]:
import itertools

In [391]:
ws=list(jumps.keys())
combinations=list(itertools.product(ws, ws))

In [392]:
matrix_form(jumps,combinations)

[((-1+0j), (-1+0j)),
 ((-1+0j), (1-0j)),
 ((-1+0j), 0),
 ((1-0j), (-1+0j)),
 ((1-0j), (1-0j)),
 ((1-0j), 0),
 (0, (-1+0j)),
 (0, (1-0j)),
 (0, 0),

In [320]:
def func(sx,sp):
    return sx*sp

In [321]:
%timeit func(sp,sz)

6.19 µs ± 255 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [322]:
a=jit(func)

In [327]:
%timeit a(sp,sz)

8.32 µs ± 316 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
